# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:  Javier Sánchez Buezas<br>
Url: https://github.com/JavierSanchezB/03MAIR---Algoritmos-de-Optimizacion/blob/main/SEMINARIO/<br>
Problema:
>2. Organizar los horarios de partidos de La Liga<br>


Descripción del problema:(copiar enunciado)

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de
liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la
audiencia.

(*) La respuesta es obligatoria
                        

## (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



## ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

Sin tener en cuenta las restricciones hay n_partidos ^ n_horarios posibilidades. En este caso es 10^10.

# Modelo para el espacio de soluciones
## (*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

La mejor estructura para mí ha sido una lista de longitud igual al número de partidos en la que el elemento i-esimo es el horario asignado al partido i-esimo.

Esta estructura es fácil de iterar y se acopla bien con el resto de utilidades del programa.

En base a esa ordenación de horarios para los partidos dados, el cálculo de la audiencia de una solución es una cuestión básica de buscar coeficientes en tablas y multiplicar.

## Según el modelo para el espacio de soluciones<br>
## (*)¿Cual es la función objetivo?

## (*)¿Es un problema de maximización o minimización?

Respuesta

La función objetivo es la suma de las audiencias de cada partido, teniendo en cuenta las penalizaciones de cada uno por horario o coincidencias.

Es un problema de maximización, queremos la mayor audicencia posible para unos partidos dados.

## Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [1]:
#####################################
# IMPORTS
#####################################
import numpy as np

from pprint import pprint
from itertools import permutations

#####################################
# DATOS
#####################################
EQUIPOS = ['Celta', 'Real Madrid', 'Valencia', 'Real Sociedad', 'Mallorca', 'Eibar',
          'Athletic', 'Barcelona', 'Leganés', 'Osasuna', 'Villareal', 'Granada',
          'Alavés', 'Levante', 'Espanyol', 'Sevilla', 'Betis', 'Valladolid',
          'Atlético', 'Getafe']

PARTIDOS = [('Celta', 'Real Madrid'),
           ('Valencia', 'Real Sociedad'),
           ('Mallorca', 'Eibar'),
           ('Athletic', 'Barcelona'),
           ('Leganés', 'Osasuna'),
           ('Villareal', 'Granada'),
           ('Alavés', 'Levante'),
           ('Espanyol', 'Sevilla'),
           ('Betis', 'Valladolid'),
           ('Atlético', 'Getafe')]

CATEGORIAS = { 'Celta' : 'B',
             'Real Madrid' : 'A',
             'Valencia' : 'B',
             'Real Sociedad' : 'A',
             'Mallorca' : 'C',
             'Eibar' : 'C',
             'Athletic' : 'B',
             'Barcelona' : 'A',
             'Leganés' : 'C',
             'Osasuna' : 'C',
             'Villareal' : 'B',
             'Granada' : 'C',
             'Alavés' : 'B',
             'Levante' : 'B',
             'Espanyol' : 'B',
             'Sevilla' : 'B',
             'Betis' : 'B',
             'Valladolid' : 'C',
             'Atlético' : 'B',
             'Getafe' : 'B',}

HORARIOS = {'V20' : 0.4,
            'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8, 'S20' : 1.0,
            'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85, 'D20' : 1.0,
            'L20' : 0.4}

AUDIENCIAS = { 'AA' : 2, 'AB' : 1.3, 'AC' : 1,
              'BB' : 0.9, 'BC' : 0.75,
              'CC' : 0.47}

COINCIDENCIAS = [0, .25, .45, .60, .70, .75, .78, .80, .80]

#####################################
# FUNCIONES AUXILIARES
#####################################
def audiencia_base(partidos):
    """
    Calcula la base de audiencia de todos los partidos dados.
    
    Args:
        partidos ():
        
    Returns:
        audiencia ():
    """
    audiencia = []
    for partido in partidos:
        categoria = CATEGORIAS.get(partido[0]) + CATEGORIAS.get(partido[1])
        # Esto hace que una categoría y su inversa sean la misma
        if(base := AUDIENCIAS.get(categoria)):
            audiencia.append(base)
        else:
            audiencia.append(AUDIENCIAS.get(categoria[::-1]))
            
    return audiencia


def calcular_audiencia_solucion(solucion, verbose=False):
    """
    Calcula la audiencia total de una solución dada.
    
    Args:
        solución (dict): Diccionario {partido : horario} con las asignaciones.
        verbose (bool): Mostrar o no tabla con los valores.
        
    Returns:
        audiencia (float): Audiencia total de la distribución sugerida.
    """
    partidos = solucion.keys()
    horarios = list(solucion.values())
    base = audiencia_base(partidos)
    ponderacion = [HORARIOS.get(horario) for horario in horarios]
    correccion_coincidencia = calcular_coincidencias(horarios)
    audiencia = np.multiply(base, ponderacion)
    audiencia = np.multiply(audiencia, correccion_coincidencia)
    
    return audiencia


def calcular_coincidencias(horarios):
    """
    Calcula los coeficientes de corrección de coincidencia de una solución dada.
    
    Args:
        horarios (list): Lista con los horarios.
        
    Returns:
        coincidencias (list): Lista con los coeficientes de corrección de cada horario.
    """
    uniques = set(horarios)
    cuentas = [horarios.count(horario) for horario in uniques]
    coef = [1 - COINCIDENCIAS[c - 1] for c in cuentas]
    coef = dict(zip(uniques, coef))
    coincidencias = [coef.get(h) for h in horarios]
    
    return coincidencias


def calcular_todos_horarios(partido, sol):
    """
    Calcula la audiencia de un partido sin asignar en una solución parcial.
    
    Args:
        partido (tuple): Tupla con los nombres de los equipos.
        
        sol (dict): Solución parcial en la que insertar el partido.
        
    Returns:
        audiencias (disct): Audiencia de la solución
            con el partido añadido en todos los horarios posibles
    """
    audiencias = []
    for horario in HORARIOS.keys():
        parcial = sol.copy()
        parcial[partido] = horario
        parcial = {a : b for a, b in parcial.items() if not b == ''}
        audiencia_parcial = calcular_audiencia_solucion(parcial)
        audiencias.append(sum(audiencia_parcial))

    return audiencias

In [2]:
# Genera todas las permutaciones de tamaño 10 de los horarios posibles
h = permutations(HORARIOS.keys(), 10)

maxima = 0
# Itera en las permutaciones buscando el máximo de audiencia
for horario in h:
    # Limita los cálculos a los horarios que cumplen las restricciones
    if 'V20' in horario and 'L20' in horario:
        audiencia = calcular_audiencia_solucion(dict(zip(PARTIDOS, horario)))
        if sum(audiencia) > maxima:
            maxima = sum(audiencia)

print(f'Máxima audiencia posible: {maxima}')

Máxima audiencia posible: 6.856


## Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

La complejidad del algoritmo por fuerza bruta es: O(n^n), recorre todas las posibles soluciones.

## (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [3]:
PARTIDOS = [('Celta', 'Real Madrid'),
           ('Valencia', 'Real Sociedad'),
           ('Mallorca', 'Eibar'),
           ('Athletic', 'Barcelona'),
           ('Leganés', 'Osasuna'),
           ('Villareal', 'Granada'),
           ('Alavés', 'Levante'),
           ('Espanyol', 'Sevilla'),
           ('Betis', 'Valladolid'),
           ('Atlético', 'Getafe')]

# Calculamos la base de todos los partidos para establecer un orden
bases = audiencia_base(PARTIDOS)
# Lista en la que se guarda la asignación de los horarios
horarios = [''] * len(PARTIDOS)

# Se buscan el máximo y los dos mínimos
# El máximo se asigna al horario de mayor audiencia
# Los dos mínimos se asignan a los dos horarios de menor audiencia.
# Esta tambien hace que se cumpla la restricción de un partido cada Viernes y Lunes
index_max = np.argmax(bases)
base_min_1 = np.sort(bases)[0]
base_min_2 = np.sort(bases)[1]

if base_min_1 == base_min_2:
    index_min_1 = bases.index(base_min_1)
    index_min_2 = len(bases) - bases[::-1].index(base_min_2) - 1
else:
    index_min_1 = bases.index(base_min_1)
    index_min_2 = bases.index(base_min_2)

# Asignamos el partido con mayor base al mejor horario
# y los dos con los peores horarios a los horarios impuestos (los peores)
horarios[index_max] = 'D20'
horarios[index_min_1] = 'V20'
horarios[index_min_2] = 'L20'

# Iteramos hasta que todos los partidos tengan un horario asignado
while '' in horarios:
    # Índices (de entre todos los partidos) de cuales quedan por asignar
    index_pendientes = [i for i in range(len(horarios)) if horarios[i] == '']
    
    # Lista de los partidos pendientes
    partidos_pendientes = [PARTIDOS[i] for i in index_pendientes]
    
    # Audiencia base de los partidos que quedan
    bases_pendientes = audiencia_base(partidos_pendientes)
    
    # Índice (de entre todos los partidos) del partido con máxima audiencia base
    partido_max = index_pendientes[np.argmax(bases_pendientes)]
    
    # Calcula la audiencia que se obtendría poniendo el partido
    # que estamos tratando en cada posible horario
    opciones = calcular_todos_horarios(PARTIDOS[partido_max],
                                       dict(zip(PARTIDOS, horarios)))
    # Nos quedamos con el horario que mejor resultado ha dado
    mejor_horario = np.argmax(opciones)
    
    # Lo ponemos en la lista de los horarios
    horarios[partido_max] = list(HORARIOS.keys())[mejor_horario]
    
solucion = calcular_audiencia_solucion(dict(zip(PARTIDOS, horarios)))
print('Los horarios propuestos son:')
pprint(dict(zip(PARTIDOS, horarios)))
print(f'La audiencia total de la solución propuesta es: {sum(solucion)}')

Los horarios propuestos son:
{('Alavés', 'Levante'): 'S18',
 ('Athletic', 'Barcelona'): 'D18',
 ('Atlético', 'Getafe'): 'S16',
 ('Betis', 'Valladolid'): 'D12',
 ('Celta', 'Real Madrid'): 'D20',
 ('Espanyol', 'Sevilla'): 'D16',
 ('Leganés', 'Osasuna'): 'L20',
 ('Mallorca', 'Eibar'): 'V20',
 ('Valencia', 'Real Sociedad'): 'S20',
 ('Villareal', 'Granada'): 'S12'}
La audiencia total de la solución propuesta es: 6.856


## (*)Calcula la complejidad del algoritmo 

Respuesta

Lo voy a desglosar por funciones:
 * audiencia_base:
       La complejidad de esta función es de O(n).
       Simplemente recorre la lista de n elementos haciendo un par de operaciones simples por cada uno.
 * calcular_coincidencias
       La complejidad de esta función es de O(n).
       Igual que el anterior, son asignaciones y operaciones básicas en la lista de n elementos.
 * calcular_audiencia_solucion:
       La complejidad de esta función es de O(n).
       En esencia es la asignación y multiplicación de 3 listas de n elementos.
 * calcular_todos_horarios
       La complejidad de esta función es de O(n^2).
       Recorre la lista de n elementos llamando a la función calcular_audiencia_solucion.
       
Por todo esto, el bucle while principal tiene una complejidad de O(n^3), ya que recorre la lista de n elementos llamando a la función calcular_todos_horarios.

## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [4]:
np.random.shuffle(EQUIPOS)

PARTIDOS = []
i = 0
while i < len(EQUIPOS) - 1:
    PARTIDOS.append((EQUIPOS[i], EQUIPOS[i + 1]))
    i += 2
    
print('Partidos aleatorios:')
pprint(PARTIDOS)
print()

Partidos aleatorios:
[('Sevilla', 'Getafe'),
 ('Villareal', 'Espanyol'),
 ('Mallorca', 'Barcelona'),
 ('Leganés', 'Eibar'),
 ('Atlético', 'Athletic'),
 ('Real Madrid', 'Granada'),
 ('Osasuna', 'Levante'),
 ('Valencia', 'Real Sociedad'),
 ('Alavés', 'Celta'),
 ('Betis', 'Valladolid')]



## Aplica el algoritmo al juego de datos generado

Respuesta

In [5]:
# Calculamos la base de todos los partidos para establecer un orden
bases = audiencia_base(PARTIDOS)
# Lista en la que se guarda la asignación de los horarios
horarios = [''] * len(PARTIDOS)

# Se buscan el máximo y los dos mínimos
# El máximo se asigna al horario de mayor audiencia
# Los dos mínimos se asignan a los dos horarios de menor audiencia.
# Esta tambien hace que se cumpla la restricción de un partido cada Viernes y Lunes
index_max = np.argmax(bases)
base_min_1 = np.sort(bases)[0]
base_min_2 = np.sort(bases)[1]

if base_min_1 == base_min_2:
    index_min_1 = bases.index(base_min_1)
    index_min_2 = len(bases) - bases[::-1].index(base_min_2) - 1
else:
    index_min_1 = bases.index(base_min_1)
    index_min_2 = bases.index(base_min_2)

# Asignamos el partido con mayor base al mejor horario
# y los dos con los peores horarios a los horarios impuestos (los peores)
horarios[index_max] = 'D20'
horarios[index_min_1] = 'V20'
horarios[index_min_2] = 'L20'

# Iteramos hasta que todos los partidos tengan un horario asignado
while '' in horarios:
    # Índices (de entre todos los partidos) de cuales quedan por asignar
    index_pendientes = [i for i in range(len(horarios)) if horarios[i] == '']
    
    # Lista de los partidos pendientes
    partidos_pendientes = [PARTIDOS[i] for i in index_pendientes]
    
    # Audiencia base de los partidos que quedan
    bases_pendientes = audiencia_base(partidos_pendientes)
    
    # Índice (de entre todos los partidos) del partido con máxima audiencia base
    partido_max = index_pendientes[np.argmax(bases_pendientes)]
    
    # Calcula la audiencia que se obtendría poniendo el partido
    # que estamos tratando en cada posible horario
    opciones = calcular_todos_horarios(PARTIDOS[partido_max],
                                       dict(zip(PARTIDOS, horarios)))
    # Nos quedamos con el horario que mejor resultado ha dado
    mejor_horario = np.argmax(opciones)
    
    # Lo ponemos en la lista de los horarios
    horarios[partido_max] = list(HORARIOS.keys())[mejor_horario]
    
solucion = calcular_audiencia_solucion(dict(zip(PARTIDOS, horarios)))
print('Los horarios propuestos son:')
pprint(dict(zip(PARTIDOS, horarios)))
print(f'La audiencia total de la solución propuesta es: {sum(solucion)}')

Los horarios propuestos son:
{('Alavés', 'Celta'): 'S12',
 ('Atlético', 'Athletic'): 'S16',
 ('Betis', 'Valladolid'): 'D12',
 ('Leganés', 'Eibar'): 'V20',
 ('Mallorca', 'Barcelona'): 'S20',
 ('Osasuna', 'Levante'): 'L20',
 ('Real Madrid', 'Granada'): 'D18',
 ('Sevilla', 'Getafe'): 'S18',
 ('Valencia', 'Real Sociedad'): 'D20',
 ('Villareal', 'Espanyol'): 'D16'}
La audiencia total de la solución propuesta es: 6.4955


## Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

## Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta